In [2]:
import pandas as pd
import numpy as np
from cataclop.ml.pipeline import factories
from cataclop.ml.exploration import random_race
from cataclop.ml import preprocessing

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

NAN_FLAG = 10000

In [3]:
#d = factories.Dataset.factory('default', {"from": '2013-05-10'})
d = factories.Dataset.factory('default', {"from": '2018-05-10'})
d.load(force=True)

In [4]:
d.players.head()

,age,earnings,final_odds,final_odds_ref,handicap_distance,handicap_weight,herder_id,horse_id,imported_at_player,is_first_timer,is_racing,jockey_id,music,num,owner_id,placed_2_count,placed_3_count,placed_count,placed_dividend,placed_earnings,position,post_position,prev_year_earnings,race_count,race_id,time,trainer_id,victory_count,victory_earnings,winner_dividend,year_earnings,category,condition_age,condition_sex,declared_player_count,distance,imported_at_race,num_race,num_bis,prize,session_id,start_at,sub_category,date,hippodrome_id,imported_at_session,num_session,code,country,imported_at_hippo,name,hist_1_pos,hist_2_pos,hist_3_pos,hist_4_pos,hist_5_pos,hist_6_pos,speed
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
131164,2,540000,6.6,7.6,NaN,NaN,434.0,51010,2018-06-22 18:31:08.296166,False,True,2239,3p3p,1,1262,0,2,2,0.0,540000,5.0,5,0,2,18678,NaN,517,0,540000,0.0,540000,PLAT,DEUX_ANS,TOUS_CHEVAUX,7,1000,2018-06-22 18:31:08.208634,1,NaN,18000,2949,2018-05-10 13:37:00,COURSE_A_CONDITIONS,2018-05-10,44,2018-06-22 18:31:08.204881,1,LYO,FRA,2018-05-29 01:17:30.716760,LYON-PARILLY,3.0,3.0,0.0,0.0,0.0,0.0,1000.0
131165,2,270000,12.8,14.5,NaN,NaN,1134.0,51011,2018-06-22 18:31:08.393732,False,True,55,3p6p,2,792,0,1,1,0.0,270000,6.0,4,0,2,18678,NaN,516,0,270000,0.0,270000,PLAT,DEUX_ANS,TOUS_CHEVAUX,7,1000,2018-06-22 18:31:08.208634,1,NaN,18000,2949,2018-05-10 13:37:00,COURSE_A_CONDITIONS,2018-05-10,44,2018-06-22 18:31:08.204881,1,LYO,FRA,2018-05-29 01:17:30.716760,LYON-PARILLY,3.0,6.0,0.0,0.0,0.0,0.0,1000.0
131166,2,0,24.3,18.9,NaN,NaN,1171.0,52306,2018-06-22 18:31:08.493890,False,True,39,8p,3,7566,0,0,0,0.0,0,7.0,7,0,1,18678,NaN,415,0,0,0.0,0,PLAT,DEUX_ANS,TOUS_CHEVAUX,7,1000,2018-06-22 18:31:08.208634,1,NaN,18000,2949,2018-05-10 13:37:00,COURSE_A_CONDITIONS,2018-05-10,44,2018-06-22 18:31:08.204881,1,LYO,FRA,2018-05-29 01:17:30.716760,LYON-PARILLY,8.0,0.0,0.0,0.0,0.0,0.0,1000.0
131167,2,405000,5.0,5.2,NaN,NaN,8743.0,50868,2018-06-22 18:31:08.585531,False,True,62,3p,4,1723,0,1,1,0.0,405000,4.0,2,0,1,18678,NaN,75,0,405000,0.0,405000,PLAT,DEUX_ANS,TOUS_CHEVAUX,7,1000,2018-06-22 18:31:08.208634,1,NaN,18000,2949,2018-05-10 13:37:00,COURSE_A_CONDITIONS,2018-05-10,44,2018-06-22 18:31:08.204881,1,LYO,FRA,2018-05-29 01:17:30.716760,LYON-PARILLY,3.0,0.0,0.0,0.0,0.0,0.0,1000.0
131168,2,0,40.4,37.9,NaN,NaN,55.0,53514,2018-06-22 18:31:08.910177,True,True,135,Inédit,5,227,0,0,0,800.0,0,2.0,3,0,0,18678,NaN,517,0,0,0.0,0,PLAT,DEUX_ANS,TOUS_CHEVAUX,7,1000,2018-06-22 18:31:08.208634,1,NaN,18000,2949,2018-05-10 13:37:00,COURSE_A_CONDITIONS,2018-05-10,44,2018-06-22 18:31:08.204881,1,LYO,FRA,2018-05-29 01:17:30.716760,LYON-PARILLY,0.0,0.0,0.0,0.0,0.0,0.0,1000.0


In [ ]:
# keep the races we are interested in
df = d.players.groupby('race_id').filter(lambda race: True)
#df = d.players.groupby('race_id').filter(lambda race: race['race_count'].max() > 0)

races = df.groupby('race_id')

In [ ]:
# append average speed, might be used as a target
df['speed'] = (df['distance'] / df['time']).fillna(1000)

# 
df['win'] = (df['position'] == 1).astype(np.float)

#
df['victory_earnings'] = np.log(1+df['victory_earnings'].fillna(0))
df['placed_earnings'] = np.log(1+df['placed_earnings'].fillna(0))
df['prev_year_earnings'] = np.log(1+df['prev_year_earnings'].fillna(0))
df['year_earnings'] = np.log(1+df['year_earnings'].fillna(0))

# append last odds inverse, equivalent to the estimated probability of winning 
df['final_odds_ref_inv'] = (1. / df['final_odds_ref']).fillna(0.)

In [ ]:
# features we want to append stats relative to the race (mean, std ...)
agg_features = ['race_count', 
                'victory_count', 
                'placed_2_count', 
                'placed_3_count',
                'victory_earnings',
                'placed_earnings',
                'prev_year_earnings',
                'handicap_distance',
                'handicap_weight'
               ]
stats = races[agg_features].agg([np.mean, np.std, np.min, np.max])
# transform the 2 level columns index in 1 (victory_count, (mean, std)) -> (victory_count_mean, victory_count_std) 
stats.columns = ['_'.join(col) for col in stats.columns.values]

In [ ]:
df = df.join(stats, how='left', on='race_id')

for f in agg_features:
    df['{}_r'.format(f)] = (df[f] - df['{}_mean'.format(f)]) / df['{}_std'.format(f)]
    
relative_features = ['{}_r'.format(f) for f in agg_features]

df[relative_features] = df[relative_features].replace([np.inf, -np.inf], np.nan)
df[relative_features] = df[relative_features].fillna(1000)

In [ ]:
# append sorted odds of the race
odds = pd.DataFrame(columns=['odds_{:d}'.format(i) for i in range(20)], index=df.index)

races = df.groupby('race_id')
for (id, race) in races:
    odds_sorted = sorted(race['final_odds_ref'].values)[0:20]
    odds.loc[race.index, ['odds_{:d}'.format(i) for i, v in enumerate(odds_sorted)]] = odds_sorted

df = pd.concat([df,odds], axis=1)

df[['odds_{:d}'.format(i) for i in range(20)]] = df[['odds_{:d}'.format(i) for i in range(20)]].fillna(1000.0)

In [ ]:
# keep only interesting races: at least one winner, with reference odds
df = df.groupby('race_id').filter(lambda r: r['position'].min() == 1 and r['odds_0'].min() != 1000 )
df = df.sort_index()

In [ ]:
random_race(df, cols=['position', 'sub_category', 'num', 'final_odds', 'final_odds_ref', 'odds_0', 'odds_1', 'odds_2'] + agg_features, n=1)

In [ ]:
features = ['prize', 'declared_player_count']

#features += ['final_odds_ref']

features += ['odds_{:d}'.format(i) for i in range(10)]

features += ['hist_{}_pos'.format(i+1) for i in range(6)]

features += agg_features

# too lazy to write a comprehension
for f in agg_features:
    for s in ['amin', 'amax', 'std', 'mean', 'r']:
        features.append('{}_{}'.format(f, s))
        
# unique features list
features = list(set(features))

# this will be transformed in the model training
categorical_features = ['category', 'sub_category', 'country']

In [ ]:
sorted(features)

In [ ]:
N_TARGETS = 1

df['target'] = df['winner_dividend'] / 100.
#df.loc[df['target'] > N_TARGETS, 'target'] = np.log(1.+N_TARGETS)
df['target'].fillna(0, inplace=True)

# to test the training is working we can append the target 
#features.append('target')

#for f in features:
#    print('{} \t: {}'.format(f, df[f].corr(df['target'])))

In [ ]:
from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import log_loss, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

# 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier, MLPRegressor

groups = df['race_id'].values

group_kfold = GroupKFold(n_splits=2)

splits = list(group_kfold.split(df.values, df['position'].values, groups))

models = [
    {
        'name': 'knn_10',
        'model': KNeighborsRegressor(n_neighbors=10),
        'scaler': True
    },
    {
        'name': 'knn_30',
        'model': KNeighborsRegressor(n_neighbors=30),
        'scaler': True
    },
    {
        'name': 'mlp',
        'model': MLPRegressor(),
        'scaler': True
    },
    {
        'name': 'rf_10',
        'model': RandomForestRegressor(n_estimators=10),
        'scaler': False
    },
    {
        'name': 'rf_100',
        'model': RandomForestRegressor(n_estimators=100),
        'scaler': False
    },
]

for model in models:
    for i in range(N_TARGETS):
        df['pred_{}_{}'.format(model['name'], i+1)] = 0.0


In [ ]:
df.loc[:, features] = df.loc[:, features].fillna(NAN_FLAG)
df = df.reset_index()

In [ ]:
for train_index, test_index in splits:
    
    for model in models:

        X_train = df[features].iloc[train_index].copy()
        y_train = df['target'].iloc[train_index]

        dummies = preprocessing.get_dummies(df.iloc[train_index], categorical_features)
        X_train = pd.concat([X_train, preprocessing.get_dummy_values(df.iloc[train_index], dummies)], axis=1)

        idx = (df.iloc[train_index]['target'] != NAN_FLAG) & (df.iloc[train_index]['category'] == 'PLAT') & (df.iloc[train_index]['final_odds_ref'] < 30) & ((df.iloc[train_index]['position'] == 1) | (df.iloc[train_index]['position'] == 4)) 
        X_train = X_train[ idx ]
        y_train = df['target'].iloc[train_index][ idx ]

        X_test = df[features].iloc[test_index].copy()
        y_test = df['target'].iloc[test_index]
    
        X_test = pd.concat([X_test, preprocessing.get_dummy_values(df.iloc[test_index], dummies)], axis=1)
    
        X_train = X_train.values
        X_test = X_test.values
    
        idx = df.iloc[test_index].index

        if model['scaler']:
            scaler = StandardScaler()
            scaler.fit(X_train)

            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)

        clf = model['model']
        clf.fit(X_train, y_train.values)
        
        if N_TARGETS > 1:
            
            p = clf.predict_proba(X_test)

            print( model['name'], log_loss(y_test.values, p) )

            for i in range(N_TARGETS):
                df.loc[idx, 'pred_{}_{}'.format(model['name'], i+1)] = p[:, list(clf.classes_).index(i+1)]
            
        else:

            p = clf.predict(X_test)
        
            df.loc[idx, 'pred_{}_1'.format(model['name'])] = p
        
            print( model['name'], mean_absolute_error(y_test.values, p) )


In [ ]:
random_race(df, ['sub_category', 'num', 'position', 'final_odds', 'pred_rf_10_1', 'pred_rf_100_1', 'pred_knn_10_1', 'pred_knn_30_1', 'pred_mlp_1'])

In [ ]:
for model in models:
    clf = model['model']
    if hasattr(clf, 'feature_importances_'):
        fi = pd.Series(clf.feature_importances_, index=features+preprocessing.get_dummy_features(dummies))
        print(fi.sort_values(ascending=False))
        break

In [ ]:
targets = ['pred_knn_10_1', 'pred_knn_30_1', 'pred_rf_10_1', 'pred_rf_100_1', 'pred_mlp_1']

df[targets].describe()

In [ ]:
races = df.sort_values('start_at').groupby('race_id')

bets = []

stash = 500

for (id, race) in races:
    
    candidate_bets = []
    
    nums = []
    
    for target in targets:

        r = race.sort_values(by=target, ascending=False)

        odds = r['final_odds_ref'].min()
        nth = len(r[r['final_odds_ref']<odds])+1

        N = 1

        for n in range(N):

            if len(r) <= N:
                continue

            player = r.iloc[n]

            if player['num'] in nums:
                continue

            odds = player['final_odds_ref']

            if odds > 20:
                break

            nth = (r['final_odds_ref']<odds).sum()+1

            bet = 1.

            profit = player['winner_dividend']/100.0 * bet - bet

            stash = stash + profit
            
            row = [id, player['date'], player['num'], odds, player['final_odds'], target, player[target], bet, profit]
            
            for f in features:
                row.append(player[f])
            for f in categorical_features:
                row.append(player[f])

            candidate_bets.append( row )

            nums.append(player['num'])
            
            break
    
    if len(candidate_bets) == 1:
        bets += candidate_bets
        
        
cols = ['id', 'date', 'num', 'odds_ref', 'odds_final', 'target', 'pred', 'bet', 'profit'] + features + categorical_features

bets = pd.DataFrame(bets, columns=cols)

bets.index = bets['date']

bets = bets.sort_index()

bets['bets'] = bets['bet'].cumsum()
bets['stash'] = bets['profit'].cumsum()

In [ ]:
bets[(bets['target'] == 'pred_rf_100_1') & (bets['odds_ref'] > 5) & (bets['category'] == 'PLAT')][['num', 'odds_final', 'pred', 'profit', '']]

In [ ]:
bets

In [ ]:
bets[(bets['target'] == 'pred_rf_10_1') & (bets['odds_ref'] > 1) & (bets['category'] != 'PLAT')]['profit'].describe()

In [ ]:
len(bets)